# Grupo 3: Trabajo Final Integrador

Churichi Alan, Gonzalez Exequiel, Mare Rodrigo

In [1]:
import roboticstoolbox as rtb
import numpy as np
import sim
import sys
import time
from handlers import *
from kinematics import *
from utils import *
from trajectories import *
from constants import *

### Movimiento del robot utilizando el MCI calculado

In [2]:
clientID = connect(19999)

t0 = 0;
if (clientID != -1): # Mientras haya conexión con Coppelia
    joints = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    home = {}
    checkpoints = [{} for i in range(CHECKPOINTS_NUM)]
    
    # Se obtienen los IDs
    joint_handlers(joints, JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints, 'CheckPoint', CHECKPOINTS_NUM, clientID, sim.simx_opmode_blocking)
    _, tip = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip'), sim.simx_opmode_blocking)

    
    # Se setean las posiciones y orientaciones como streaming
    sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip, -1, sim.simx_opmode_streaming)
    for i in range(CHECKPOINTS_NUM):
        sim.simxGetObjectPosition(clientID, checkpoints[i]['handle'], -1, sim.simx_opmode_streaming)
        sim.simxGetObjectOrientation(clientID, checkpoints[i]['handle'], -1, sim.simx_opmode_streaming)

    # Se inicia la simulación
    sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)
    
    time.sleep(1) # Tiempo de espera para asegurarse que los valores leidos de home sean correctos
    
    # Posición, orientación y ángulos de Home
    error, home['position'] = sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_buffer)
    error, home['orientation'] = sim.simxGetObjectOrientation(clientID, tip, -1, sim.simx_opmode_buffer)
    theta = inverse_kinematics(home['position'], home['orientation'])
    if(not theta):  
        raise Exception("Posición de Home no permitida")
    else:
        home['angles'] = theta
    
    # Posición, orientación y ángulos de Checkpoints
    for i in range(CHECKPOINTS_NUM):
        error, checkpoints[i]['position'] = sim.simxGetObjectPosition(clientID, 
                                                                      checkpoints[i]['handle'], 
                                                                      -1, 
                                                                      sim.simx_opmode_buffer)
        error, checkpoints[i]['orientation']  = sim.simxGetObjectOrientation(clientID, 
                                                                             checkpoints[i]['handle'], 
                                                                             -1, 
                                                                             sim.simx_opmode_buffer)
        theta = inverse_kinematics(checkpoints[i]['position'], checkpoints[i]['orientation'])
        if(not theta):  
            raise Exception(f"Posición de Checkpoint {i + 1} no permitida")
        else:
            checkpoints[i]['angles'] = theta
        
    points_fast_translation = 50  # Movimiento rápido
    points_mid_translation = 150  # Movimiento moderado
    points_slow_translation = 250 # Movimiento lento
    
    print('Calculando trayectorias...')
    
    traj_1 = get_angles_for_trajectory([home, checkpoints[0], checkpoints[1]],
                                       ['jtraj', 'ctraj'], 
                                       points_fast_translation)
    traj_2 = get_angles_for_trajectory([checkpoints[1], checkpoints[0], checkpoints[2], checkpoints[3], checkpoints[4]],
                                       ['ctraj', 'ctraj', 'ctraj', 'ctraj'], 
                                       points_slow_translation)
    traj_3 = get_angles_for_trajectory([checkpoints[4], checkpoints[3], checkpoints[2], checkpoints[5], checkpoints[6]],
                                       ['ctraj', 'jtraj', 'jtraj', 'ctraj'], 
                                       points_mid_translation)
    traj_4 = get_angles_for_trajectory([checkpoints[6], checkpoints[5], checkpoints[2], checkpoints[0], checkpoints[1]],
                                       ['ctraj', 'ctraj', 'ctraj', 'ctraj'], 
                                       points_slow_translation)
    traj_5 = get_angles_for_trajectory([checkpoints[1], checkpoints[0], home],
                                       ['ctraj', 'jtraj'], 
                                       points_fast_translation)
    
    print('Moviendo...')
    
    if (sim.simxGetConnectionId(clientID) != -1): # Se ejecuta la sencuencia de movimientos
        for traj_point in traj_1:
            time.sleep(0.05)
            setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
        sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar
        
        for traj_point in traj_2:
            time.sleep(0.1)
            setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
        sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar
            
        for traj_point in traj_3:
            time.sleep(0.05)
            setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
        sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar
        
        for traj_point in traj_4:
            time.sleep(0.1)
            setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
        sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar
            
        for traj_point in traj_5:
            time.sleep(0.05)
            setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

    sim.simxFinish(clientID);
    print('Conexión Finalizada')



Conectado a  19999
Calculando trayectorias...
Moviendo...
Conexión Finalizada
